In [2]:
import os
import math
from functools import partial

import numpy as np
import pandas as pd
import torch
import re
import nltk
import torch.nn as nn
import pytorch_lightning as pl
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from transformers import DataCollatorWithPadding, AdamW, get_scheduler, set_seed
from datasets import load_metric, Dataset
from sklearn.model_selection import StratifiedKFold

In [3]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

### Configuration for training

In [4]:
class MODEL_EVAL_METRIC:
    accuracy = "accuracy"
    f1_score = "f1_score"

class Config:
    MAX_LENGTH=512
    GRADIENT_ACCUMULATION_STEPS = 1
    TWEET_COL = "text"
    RANDOM_STATE = 42
    BATCH_SIZE = 16
    OUT_SIZE = 2
    NUM_FOLDS = 5
    NUM_EPOCHS = 4
    NUM_WORKERS = 8
    TRANSFORMER_CHECKPOINT = "bert-base-uncased"
    # The hidden_size of the output of the last layer of the transformer model used
    TRANSFORMER_OUT_SIZE = 768
    PAD_TOKEN_ID = 0
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    MODEL_EVAL_METRIC = MODEL_EVAL_METRIC.accuracy
    FAST_DEV_RUN = False    
    PATIENCE = 5        
    # model hyperparameters
    MODEL_HPARAMS = {
        "learning_rate": 2e-5,
        "adam_epsilon": 1e-8,
        "weight_decay": 0.0,
        "warmup_steps": 0
    }

DATA_PATH = "./data/"

# For results reproducibility 
# sets seeds for numpy, torch, python.random and PYTHONHASHSEED.
set_seed(Config.RANDOM_STATE)

### Load the data

In [5]:
df_train = pd.read_csv('./data/train.csv')
df_test = pd.read_csv('./data/test.csv')
print(f"Rows in train.csv = {len(df_train)}")
print(f"Rows in test.csv = {len(df_test)}")
pd.set_option('display.max_colwidth', None)
df_train.head()

Rows in train.csv = 7613
Rows in test.csv = 3263


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


### K Fold CV
Split the training dataframe into kfolds for cross validation. We do this before any processing is done
on the data. We use stratified kfold if the target distribution is unbalanced

In [6]:
def strat_kfold_dataframe(df, num_folds=5):
    # we create a new column called kfold and fill it with -1
    df["kfold"] = -1
    # randomize of shuffle the rows of dataframe before splitting is done
    df.sample(frac=1, random_state=Config.RANDOM_STATE).reset_index(drop=True)
    y = df["target"].values
    skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=Config.RANDOM_STATE)
    # stratification is done on the basis of y labels, a placeholder for X is sufficient
    for fold, (train_idx, val_idx) in enumerate(skf.split(X=df, y=y)):
        df.loc[val_idx, "kfold"] = fold
    return df

df_train = strat_kfold_dataframe(df_train, num_folds=Config.NUM_FOLDS)            

### Tweet preprocessing

In [7]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/bk_anupam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/bk_anupam/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~`" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
def clean_special_chars(text, punct):
    for p in punct:
        text = text.replace(p, ' ')
    return text

def process_tweet(df, text, keyword):
    lemmatizer = WordNetLemmatizer()    
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)    
    processed_text = []
    stop = stopwords.words("english")
    for tweet, keyword in zip(df[text], df[keyword]):
        tweets_clean = []        
        # remove stock market tickers like $GE        
        tweet = re.sub(r'\$\w*', '', tweet)
        # remove old style retweet text "RT"
        tweet = re.sub(r'^RT[\s]+', '', tweet)
        # remove hyperlinks
        tweet = re.sub(r'http\S+', '', tweet)
        # remove hashtags
        # only removing the hash #, @, ... sign from the word
        tweet = re.sub(r'\.{3}|@|#', '', tweet)    
        tweet = clean_special_chars(tweet, punct)
        # remove junk characters which don't have an ascii code
        tweet = tweet.encode("ascii", "ignore").decode("utf-8", "ignore")
        # tokenize tweets        
        tweet_tokens = tokenizer.tokenize(tweet)
        for word in tweet_tokens:
            # remove stopwords and punctuation
            #if (word.isalpha() and len(word) > 2 and word not in stop and word not in string.punctuation):
                #stem_word = stemmer.stem(word)  # stemming word            
                #lem_word = lemmatizer.lemmatize(word)
                #tweets_clean.append(lem_word) 
                tweets_clean.append(word)
        processed_text.append(" ".join(tweets_clean))        
    df['processed_text'] = np.array(processed_text)

In [9]:
# Fill in missing values
df_train["keyword"] = df_train["keyword"].fillna("no_keyword")
df_test["keyword"] = df_test["keyword"].fillna("no_keyword")
process_tweet(df_train, 'text', "keyword")
process_tweet(df_test, 'text', "keyword")
# length of the processed tweet
df_train["prcsd_tweet_len"] = df_train["processed_text"].apply(lambda row: len(row.split()))
df_test["prcsd_tweet_len"] = df_test["processed_text"].apply(lambda row: len(row.split()))
df_train.iloc[50:52, :]

,id,keyword,location,text,target,kfold,processed_text,prcsd_tweet_len
50,73,ablaze,"Sheffield Township, Ohio",Deputies: Man shot before Brighton home set ablaze http://t.co/gWNRhMSO8k,1,2,deputies man shot before brighton home set ablaze,8
51,74,ablaze,India,Man wife get six years jail for setting ablaze niece\nhttp://t.co/eV1ahOUCZA,1,0,man wife get six years jail for setting ablaze niece,10


## Dataset for transformer model
Converts tweets into pytorch dataset compatible with BERT and other transformers

In [10]:
tokenizer = BertTokenizer.from_pretrained(Config.TRANSFORMER_CHECKPOINT)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [11]:
def tokenize_tweets(tokenizer, with_labels, row):
    result = tokenizer(row[Config.TWEET_COL], padding=False, truncation=True)
    if with_labels:
        result["labels"] = row["target"]
    return result

preprocess_train_data = partial(tokenize_tweets, tokenizer, True)  
preprocess_test_data = partial(tokenize_tweets, tokenizer, False)  

In [12]:
ds_train_raw = Dataset.from_pandas(df_train)
ds_test_raw = Dataset.from_pandas(df_test)

In [13]:
raw_train_col_names = ds_train_raw.column_names
raw_test_col_names = ds_test_raw.column_names

In [14]:
ds_train = ds_train_raw.map(preprocess_train_data, batched=True, remove_columns=raw_train_col_names)
ds_test = ds_test_raw.map(preprocess_test_data, batched=True, remove_columns=raw_test_col_names)


  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

### Get train and validation data for a fold

In [15]:
def get_fold_dls(fold, df):
    train_df = df[df.kfold != fold].reset_index(drop=True)
    valid_df = df[df.kfold == fold].reset_index(drop=True)
    ds_train_raw = Dataset.from_pandas(train_df)
    ds_valid_raw = Dataset.from_pandas(valid_df)
    raw_ds_col_names = ds_train_raw.column_names    
    ds_train = ds_train_raw.map(preprocess_train_data, batched=True, remove_columns=raw_ds_col_names)
    ds_valid = ds_valid_raw.map(preprocess_train_data, batched=True, remove_columns=raw_ds_col_names)
    dl_train = DataLoader(ds_train, batch_size=Config.BATCH_SIZE, shuffle=True, collate_fn=data_collator, num_workers=Config.NUM_WORKERS)
    dl_valid = DataLoader(ds_valid, batch_size=Config.BATCH_SIZE, collate_fn=data_collator, num_workers=Config.NUM_WORKERS)
    return dl_train, dl_valid, ds_train, ds_valid

In [16]:
dl_train, dl_valid, ds_train, ds_valid = get_fold_dls(0, df_train)

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [17]:
config = BertConfig.from_pretrained(Config.TRANSFORMER_CHECKPOINT, num_labels=Config.OUT_SIZE)
model = BertForSequenceClassification.from_pretrained(Config.TRANSFORMER_CHECKPOINT,config=config)
model.to(Config.DEVICE)  

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [21]:
# Optimizer
# Split weights in two groups, one with weight decay and the other not.
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": Config.MODEL_HPARAMS["weight_decay"],
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=Config.MODEL_HPARAMS["learning_rate"])

In [23]:
# Scheduler and math around the number of training steps.
num_update_steps_per_epoch = math.ceil(len(dl_train) / Config.GRADIENT_ACCUMULATION_STEPS)
num_train_steps = Config.NUM_EPOCHS * num_update_steps_per_epoch
print(f"num_update_steps_per_epoch = {num_update_steps_per_epoch}")
print(f"num_train_steps = {num_train_steps}")

num_update_steps_per_epoch = 381
num_train_steps = 1524


In [26]:
lr_scheduler = get_scheduler(
        name="linear",
        optimizer=optimizer,
        num_warmup_steps=Config.MODEL_HPARAMS["warmup_steps"],
        num_training_steps=num_train_steps,
    )

In [27]:
metric = load_metric("accuracy")

In [48]:
from tqdm.auto import tqdm

def train_fn(model, train_dataloader):
    progress_bar = tqdm(range(len(train_dataloader)))
    train_loss_epoch = []
    model.train()
    for step, batch in enumerate(train_dataloader):
        batch = {k: v.to(Config.DEVICE) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss      
        train_loss_epoch.append(loss.item())  
        loss.backward()        
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)        
    train_loss_mean = np.mean(train_loss_epoch)
    print(f"epoch {epoch+1} train_loss: {round(train_loss_mean, 4)}")            

In [49]:
def eval_fn(epoch, model, val_dataloader):
    progress_bar = tqdm(range(len(val_dataloader)))
    val_loss_epoch = []
    model.eval()    
    with torch.no_grad():
        for step, batch in enumerate(val_dataloader):    
            batch = {k: v.to(Config.DEVICE) for k, v in batch.items()}           
            outputs = model(**batch)
            val_loss_epoch.append(outputs.loss.item())
            predictions = outputs.logits.argmax(dim=-1) 
            metric.add_batch(predictions=predictions, references=batch["labels"])
            progress_bar.update(1)

    eval_metric = metric.compute()
    val_loss_mean = np.mean(val_loss_epoch)
    print(f"epoch {epoch} val_loss: {round(val_loss_mean, 4)}")
    print(f"epoch {epoch} {Config.MODEL_EVAL_METRIC}: {eval_metric}")

In [50]:
for epoch in range(Config.NUM_EPOCHS):
    print(f"Running training for epoch {epoch+1}")
    train_fn(model, dl_train)
    print(f"Running validation for epoch {epoch+1}")
    eval_fn(epoch+1, model, dl_valid)

Running training for epoch 1


  0%|          | 0/381 [00:00<?, ?it/s]

epoch 1 train_loss: 0.1628
Running validation for epoch 1


  0%|          | 0/96 [00:00<?, ?it/s]

epoch 1 val_loss: 0.5186
epoch 1 accuracy: {'accuracy': 0.8214051214707814}
Running training for epoch 2


...

epoch 3 val_loss: 0.5363
epoch 3 accuracy: {'accuracy': 0.8154957321076822}
Running training for epoch 4


  0%|          | 0/381 [00:00<?, ?it/s]

epoch 4 train_loss: 0.1306
Running validation for epoch 4


  0%|          | 0/96 [00:00<?, ?it/s]

epoch 4 val_loss: 0.5363
epoch 4 accuracy: {'accuracy': 0.8154957321076822}


In [22]:
# from tqdm.auto import tqdm

# progress_bar = tqdm(range(num_train_steps))

# model.train()
# for epoch in range(Config.NUM_EPOCHS):
#     for batch in dl_train:
#         batch = {k: v.to(Config.DEVICE) for k, v in batch.items()}
#         outputs = model(**batch)
#         loss = outputs.loss
#         loss.backward()

#         optimizer.step()
#         lr_scheduler.step()
#         optimizer.zero_grad()
#         progress_bar.update(1)

  0%|          | 0/1272 [00:00<?, ?it/s]

In [23]:
# metric= load_metric("accuracy")
# model.eval()
# for batch in dl_valid:
#     batch = {k: v.to(Config.DEVICE) for k, v in batch.items()}
#     with torch.no_grad():
#         outputs = model(**batch)

#     logits = outputs.logits
#     predictions = torch.argmax(logits, dim=-1)
#     metric.add_batch(predictions=predictions, references=batch["labels"])

# metric.compute()

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

{'accuracy': 0.570133963750985}